In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


In [2]:
(trainx, trainy), (testx, testy) = mnist.load_data()

In [3]:
trainx4d = trainx.reshape(trainx.shape[0], 28, 28, 1).astype('float32')
testx4d = testx.reshape(testx.shape[0], 28, 28, 1).astype('float32')

In [4]:
trainx4d_normalize = trainx4d / 255
testx4d_normalize = testx4d / 255

In [5]:
trainy_onehot = np_utils.to_categorical(trainy)
testy_onehot = np_utils.to_categorical(testy)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [7]:
model = Sequential()

In [8]:
model.add(Conv2D(filters=16,
                kernel_size=(5,5),
                padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [9]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [10]:
model.add(Conv2D(filters=36,
                kernel_size=(5,5),
                padding='same',
                activation='relu'))

In [11]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [12]:
model.add(Dropout(0.25))

In [13]:
model.add(Flatten())

In [14]:
model.add(Dense(128, activation='relu'))

In [15]:
model.add(Dropout(0.5))

In [16]:
model.add(Dense(10, activation='softmax'))

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1764)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               225920    
__________

In [18]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam', metrics=['accuracy'])

In [19]:
train_history = model.fit(x=trainx4d_normalize,
                         y=trainy_onehot, validation_split=0.2,
                         epochs=10, batch_size=300, verbose=2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
40s - loss: 0.4907 - acc: 0.8464 - val_loss: 0.0962 - val_acc: 0.9723
Epoch 2/10
38s - loss: 0.1401 - acc: 0.9581 - val_loss: 0.0631 - val_acc: 0.9806
Epoch 3/10
38s - loss: 0.1020 - acc: 0.9691 - val_loss: 0.0511 - val_acc: 0.9838
Epoch 4/10
36s - loss: 0.0840 - acc: 0.9754 - val_loss: 0.0467 - val_acc: 0.9858
Epoch 5/10
39s - loss: 0.0722 - acc: 0.9777 - val_loss: 0.0396 - val_acc: 0.9888
Epoch 6/10
40s - loss: 0.0650 - acc: 0.9809 - val_loss: 0.0390 - val_acc: 0.9888
Epoch 7/10
39s - loss: 0.0557 - acc: 0.9829 - val_loss: 0.0422 - val_acc: 0.9875
Epoch 8/10
38s - loss: 0.0509 - acc: 0.9845 - val_loss: 0.0339 - val_acc: 0.9895
Epoch 9/10
38s - loss: 0.0443 - acc: 0.9861 - val_loss: 0.0326 - val_acc: 0.9901
Epoch 10/10
37s - loss: 0.0420 - acc: 0.9869 - val_loss: 0.0333 - val_acc: 0.9898


In [20]:
score = model.evaluate(testx4d_normalize, testy_onehot)
print()
print('測試集準確度：', score[1])

 9984/10000 [============================>.] - ETA: 0s
測試集準確度： 0.9913


In [21]:
prediction = model.predict_classes(testx4d_normalize)

 9952/10000 [============================>.] - ETA: 0s

In [22]:
prediction[:10]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])

In [23]:
#分析預測結果
import pandas as pd
pd.crosstab(testy, prediction, 
            rownames=['label'], colnames=['predic'])

predic,0,1,2,3,4,5,6,7,8,9
label,,,,,,,,,,
0,977,0,0,0,0,0,2,1,0,0
1,0,1131,1,0,0,1,0,1,1,0
2,2,2,1024,1,1,0,0,2,0,0
3,0,0,0,1003,0,3,0,2,2,0
4,0,0,0,0,973,0,1,0,1,7
5,1,0,0,4,0,883,2,0,0,2
6,4,2,0,1,1,1,949,0,0,0
7,0,1,2,2,0,0,0,1019,1,3
8,3,0,3,2,1,1,0,2,957,5


In [24]:
#儲存整個模型，包含結構、權重、損失函數和最佳化方法
model.save("/Users/PChomeIM/pywork/SaveModel/MnistCNNModel.h5")